In [ ]:
'''
CS6140 Assignment 1 Q1.1
Growing Decision Trees from scratch
Wing Man, Kwok  
05/18/2022
'''

In [77]:
#import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split    #Library to split training and testing data

In [90]:
#Build Classification Decision Tree, initialization of parameters, compute information gain, entropy, etc
class ClassificationDecisionTree:                       
    def __init__(self, max_depth=10, min_samples=2):    #initialize object paramenters.  Assume minimum samples in each axis rectangle is 2
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.root = None

    def terminate(self, depth):                         #check if termination criteria is met
        if (depth >= self.max_depth or self.numofSamples < self.min_samples or self.numofClasslabels == 1 ):
            return True
        return False
           
    def fit(self, X, y):                                #fit dataset into the tree    
        self.root = self.build_tree(X, y)

    def compute_entropy(self, y):                       #compute root/parent node entropy for information gain as splitting measure
        entropy = 0
        bin_propability = np.bincount(y) / len(y)       #count occurence of a value which matches with index, then normalize
        for p in bin_propability:
          if p > 0:
            entropy += -p * np.log2(p)
        return entropy

    def split_datapoints_to_leftright(self, X, best_information_gain_X):
        left_index = np.argwhere(X <= best_information_gain_X).flatten()     #split data points to left if smaller than variable X of best information gain
        right_index = np.argwhere(X > best_information_gain_X).flatten()     #split data points to right if larger than variable X of best information gain
        return left_index, right_index                                       #return location(index) of the data point split into left and right correspondingly

    def compute_information_gain(self, X, y, best_information_gain_X):        #compute information gain of a branch
        parent_loss = self.compute_entropy(y)
        left_index, right_index = self.split_datapoints_to_leftright(X, best_information_gain_X)
        
        if len(left_index) == 0 or len(right_index) == 0: 
            return 0
        
        child_loss = (len(left_index) / len(y)) * self.compute_entropy(y[left_index]) + (len(right_index) / len(y)) * self.compute_entropy(y[right_index])
        return parent_loss - child_loss

    def find_max_information_gain(self, X, y, features):                      #find max information gain for each datapoint
  
        axis_aligned_rectangle_score = - 1
        axis_aligned_rectangle_feature = None
        axis_aligned_rectangle_iris_dimension = None

        for col in features:                           #for each feature, source, eg, [3 1 2 0]
  
            X_col = X[:, col]                          #extract one column of X.  Format: X[row_index, column_index]
            iris_dimensions = np.unique(X_col)         #returns the sorted unique elements of the X column.  each represents a data point of X column
   
            for iris_dimension in iris_dimensions:      #for each unique datapoint of iris species measurement
                information_gain = self.compute_information_gain(X_col, y, iris_dimension) #calculate information gain
                if information_gain > axis_aligned_rectangle_score:                        #store parameters of highest information gain
                    axis_aligned_rectangle_score = information_gain
                    axis_aligned_rectangle_feature = col
                    axis_aligned_rectangle_iris_dimension = iris_dimension

        return axis_aligned_rectangle_feature, axis_aligned_rectangle_iris_dimension
    
    def build_tree(self, X, y, depth=0):

        self.numofSamples = X.shape[0]                    #X.shape = 105, 4 (105 training dataset, 4 features)
        self.numofFeatures = X.shape[1]   
        self.numofClasslabels = len(np.unique(y))         #return number of unique labels of column y (label)

        #exit criteria
        if self.terminate(depth):                         
            predicted_label = np.argmax(np.bincount(y))   #return the max of count of elements value same as array index
            return Node(value=predicted_label)

        #iterate each data point, find the data point with maxiumn information gain
        random_features = np.random.choice(self.numofFeatures, self.numofFeatures, replace=False) #generate 4 numbers, range 0 to 4, eg [0 1 2 3], [3 2 0 1]
        best_feature, best_information_gain = self.find_max_information_gain(X, y, random_features) #locate the datapoint with best information gain

        # populate children 
        left_index, right_index = self.split_datapoints_to_leftright(X[:, best_feature], best_information_gain)
        left_child = self.build_tree(X[left_index, :], y[left_index], depth + 1)
        right_child = self.build_tree(X[right_index, :], y[right_index], depth + 1)
        return Node(best_feature, best_information_gain, left_child, right_child)

    def traverse_tree(self, x, node):                           #to compute accuracy of prediction, input testing dataset, pass each row of X, then follow index of node.feature and traverse til reaching leaf
        if node.is_leaf():                                      #then the leaf value is returned, and compare with value of y of the row
            return node.value
    
        if x[node.feature] <= node.threshold:                   #follow the logic of building tree, so now look up the x variables of the node feature and compare the value with best information gain
            return self.traverse_tree(x, node.left)
        
        return self.traverse_tree(x, node.right)
        
    def print_tree(self, node, depth):                          #print tree by pre-order traversal (root left first)
      if node.is_leaf():
          #print (depth * "\t", "node", node)                   #print nodeID to counter-check correctness
          print (depth * "\t" + "|-----", "Class", node.value)
          return node.value
      
      #print (depth * "\t", "node", node) 
      print (depth * "\t" + "|-----", "feature", node.feature, "<=", node.threshold )
      #print (depth * "\t", "node.left", node.left)
      #print (depth * "\t", "node.right", node.right)
      
      print (depth * "\t" + "|-----", "left ")
      self.print_tree(node.left, depth+1)

      print (depth * "\t" + "|-----", "right ")
      self.print_tree(node.right, depth+1)

    def predict(self, X):                                               #predict if decision tree returns a leaf node, of which value is same as true y label.
        predictions = []
        for x in X:                                                     #x represent each row of X
          predictions.append(self.traverse_tree(x, self.root))
        return np.array(predictions)
        

In [91]:
#create decision nodes and leaf nodes
class Node:                                                            
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature                                          #store feature which is found resulting best information gain
        self.threshold = threshold                                      #store the value of feature column, which results the best information gain to split dataset
        self.left = left                                                #store left child
        self.right = right                                              #store right child
        self.value = value                                              #store predicted label as a leaf node
    
    def is_leaf(self):
      if self.value is None:
        return False
      return True

In [92]:
#return accuracy of decision tree prediction
def compute_accuracy(y_truelabel, y_predicted):                                        
    accuracy = np.sum(y_truelabel == y_predicted) / y_truelabel.size
    return accuracy

In [105]:
#prepare dataset
dataset = pd.read_csv("/content/drive/My Drive/Colab Notebooks/CS6140 Assignment1/data.csv")    

In [106]:
feature_cols = ['feature1', 'feature2', 'feature3', 'feature4']
X = dataset[feature_cols]                         #Assign all feature columns into variable X
y = dataset['class']                              #Assign all target columns into variable y

X_train, X_test, y_train, y_test = train_test_split(    #split dataset into training dataset and testing dataset
    X, y, test_size=0.2, random_state=42                 #random_state parameter gives a fixed random seed
)

#fit the tree
model = ClassificationDecisionTree(max_depth=10)
model.fit(X_train.values , y_train.values)

In [107]:
#after training, fit testing values to compute accuracy
y_predicted = model.predict(X_test.values)             

print ("predicted value by decision tree\t", y_predicted)
print("true value of y\t\t\t\t", y_test.values, "\n")

accuracy = compute_accuracy(y_test, y_predicted)     
print("Accuracy:", accuracy)

predicted value by decision tree	 [2 0 0 2 1 1 1 2 1 2 0 0 1 0 0 2 0 2 1 2 0 1 1 0 1 2 1 2 0 0]
true value of y				 [2 0 0 2 1 1 1 2 2 2 0 0 1 0 0 2 0 2 1 2 0 1 1 0 1 2 1 2 0 0] 

Accuracy: 0.9666666666666667


For the below printout of decision tree, i have referenced the structure of sklearn and draw an equivalence.

"Feature x", as shown below at each decision node, represents the feature column which has been calculated for the best information gain, then a value from that feature to split dataset into left and right branches.  

Each leaf shows a "class" value for the decision.

In [108]:
#print decision tree
model.print_tree(model.root, depth=0)                    

|----- feature 3 <= 0.6
|----- left 
	|----- Class 0
|----- right 
	|----- feature 3 <= 1.7
	|----- left 
		|----- feature 2 <= 5.0
		|----- left 
			|----- feature 0 <= 4.9
			|----- left 
				|----- feature 1 <= 2.4
				|----- left 
					|----- Class 1
				|----- right 
					|----- Class 2
			|----- right 
				|----- Class 1
		|----- right 
			|----- feature 0 <= 6.0
			|----- left 
				|----- Class 1
			|----- right 
				|----- Class 2
	|----- right 
		|----- feature 2 <= 4.8
		|----- left 
			|----- feature 0 <= 5.9
			|----- left 
				|----- Class 1
			|----- right 
				|----- Class 2
		|----- right 
			|----- Class 2
